## Create additional tables for translation

In [2]:
SELECT * FROM languages;

(5 row(s) affected)

Total execution time: 00:00:00.324

code
de
zh-Hans
hi
hu
sw


**Check the extension**

In [4]:
CREATE OR REPLACE PROCEDURE translate_listing_descriptions(max_num_listings INT DEFAULT 10)
LANGUAGE plpgsql
AS $$
BEGIN
    WITH batch_to_load(id, description) AS
    (
        SELECT id, description
        FROM listings l
        WHERE NOT EXISTS (SELECT * FROM listing_translations ll WHERE ll.listing_id = l.id)
        LIMIT max_num_listings
    )
    INSERT INTO listing_translations(listing_id, language_code, description)
    SELECT b.id, l.code, (unnest(tr.translations)).TEXT
    FROM batch_to_load b
        CROSS JOIN languages l
        CROSS JOIN LATERAL azure_cognitive.translate(b.description, l.code) tr;
END;
$$;


CREATE PROCEDURE

Total execution time: 00:00:00.253

**Test translation**

In [9]:
SELECT *
FROM azure_cognitive.translate('Hello, how are you?', 'fr');

(1 row(s) affected)

Total execution time: 00:00:00.510

translations,detected_language,source_text
"{""(fr,\""Bonjour, comment vas-tu?\"",)""}","(en,0.64)",NULL


**Translate existed reviews**

In [10]:
CALL translate_listing_descriptions(10);
CALL translate_listing_descriptions(10);
CALL translate_listing_descriptions(10);

CALL

CALL

CALL

Total execution time: 00:00:41.869

In [11]:
SELECT COUNT(*) FROM listing_translations;

(1 row(s) affected)

Total execution time: 00:00:00.322

count
150


**Create new review and test**

In [12]:
CREATE OR REPLACE PROCEDURE add_listing(id INT, name VARCHAR(255), description TEXT)
LANGUAGE plpgsql
AS $$
DECLARE
listing_id INT;
BEGIN
    INSERT INTO listings(id, name, description)
    VALUES(id, name, description);

    INSERT INTO listing_translations(listing_id, language_code, description)
    SELECT id, l.code, (unnest(tr.translations)).TEXT
    FROM languages l
        CROSS JOIN LATERAL azure_cognitive.translate(description, l.code) tr;
END;
$$;

CREATE PROCEDURE

Total execution time: 00:00:00.161

In [13]:
CALL add_listing(51, 'A Beautiful Home', 'This is a beautiful home in a great location.');

CALL

Total execution time: 00:00:00.773

In [14]:
SELECT l.id, l.name, l.description, lt.language_code, lt.description AS translated_description
FROM listing_translations lt
    INNER JOIN listings l ON lt.listing_id = l.id
WHERE l.name = 'A Beautiful Home';

(5 row(s) affected)

Total execution time: 00:00:00.370

id,name,description,language_code,translated_description
51,A Beautiful Home,This is a beautiful home in a great location.,de,Dies ist ein wunderschönes Zuhause in toller Lage.
51,A Beautiful Home,This is a beautiful home in a great location.,zh-Hans,这是一个地理位置优越的美丽住宅。
51,A Beautiful Home,This is a beautiful home in a great location.,hi,यह एक महान स्थान में एक सुंदर घर है।
51,A Beautiful Home,This is a beautiful home in a great location.,hu,Ez egy gyönyörű otthon nagyszerű helyen.
51,A Beautiful Home,This is a beautiful home in a great location.,sw,Hii ni nyumba nzuri katika eneo zuri.
